In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import glob
import json
import sys,os,argparse
import warnings
warnings.filterwarnings('ignore')

import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(20,8),'axes.facecolor':'white'})

%matplotlib inline

In [ ]:
# command line args
CONFIG_FILE = '.config_ipynb'

if os.path.isfile(CONFIG_FILE):
    with open(CONFIG_FILE) as f:
        sys.argv = f.read().split()
    
parser = argparse.ArgumentParser()
parser.add_argument("--redsheet", help="Redsheet")
parser.add_argument("--batch_path", help="path to QC outputs")
parser.add_argument("--xlsx", help="xlsx output file name")
args = parser.parse_args()

# Batch QC summary report
___

In [ ]:
# load redsheet into dataframe
redsheet = args.redsheet
#redsheet = 'test/test_redsheet.csv'
manifest = pd.read_csv(f'{redsheet}', skiprows=7, sep=',', index_col='name_alias')

# select non Methylseq samples
manifest = manifest[(manifest.analysis_type=='Whole Genome Genomic')&~(manifest.eln_sample_id.str.contains('Control'))]

# set path for merged BAM QC metrics
batch_path = args.batch_path
#batch_path = 'test'

# open summary metrics file for writing
writer =   pd.ExcelWriter(f'{args.xlsx}', engine='xlsxwriter')

In [ ]:
def group_layout(fig,df, group, jitter=False):
    samples = [len(f['x']) for f in fig['data']]
    samples = samples[:df.drop_duplicates(group).shape[0]]
    samples = [(i/np.sum(samples)) for i in samples]

    ranges = []
    col_spacing = 0.005
    for i in range(len(samples)):
        if i == len(samples):
            ranges.append((samples[i-1][-1]+col_spacing),1.0)

        elif i == 0:
            ranges.append((0.0,samples[i]-col_spacing))
        else:
            ranges.append((ranges[i-1][-1]+col_spacing,ranges[i-1][-1]+samples[i]-col_spacing))

    count = 0
    axes = []
    for f in fig['layout']:
        if 'xaxis' in f:
            if f == 'xaxis':
                axes.append('xaxis0')
            else:
                axes.append(f)
    axes = sorted(axes, key=lambda x: int(x.split('xaxis')[-1]))
    axes[0] = 'xaxis'
    for f in axes:
        fig.layout[f]['domain'] = ranges[count]
        count += 1

    for i, a in enumerate(fig.layout.annotations):
        r = ranges[i]
        fig.layout.annotations[i]['x'] = r[0]+(r[-1]-r[0])/2
        fig.layout.annotations[i]['font']['size'] = 12
        fig.layout.annotations[i]['text'] = fig.layout.annotations[i]['text']#.replace('Buffer','Buf').replace('Enhancer','Enh').replace('preext','pX')
        if jitter:
            if i % 2 == 0:
                fig.layout.annotations[i]['y'] = fig.layout.annotations[i]['y']+0.05
    return fig

In [ ]:
def glob_fastp_to_df(manifest,data_path=None, skiprows=None, index_col=None, nrows=None, axis=0, add_sample_name=True, header=0,sep='\t'):
    
    input_files = []
    input_files += glob.glob(f'{data_path}/*.json')
    dfs = []
    for file in sorted(input_files):
        with open(file) as train_file:
            fastp_dict = json.load(train_file)

        dict_train = fastp_dict['filtering_result']
        sample = file.split('/')[-1].split('.')[0]
        df = pd.DataFrame.from_dict(dict_train, orient='index')
        df.columns = [sample]
        df = df.T
        df['lane'] = [l.split('_')[-1] for l in df.index]
        df['reads_before_filtering'] = fastp_dict['summary']['before_filtering']['total_reads']
        df['sample_name'] = ['_'.join(l.split('_')[:-2]) for l in df.index]
        dfs.append(df)
    df = pd.concat(dfs)
    return df

In [ ]:
def glob_to_df(glob_path, skiprows=None, index_col=None, nrows=None, axis=0, col_names=None, add_sample_name=True, normalize=False, header=0,sep='\t'):
    input_files = glob.glob(glob_path)
    dfs = []
    for input_file in input_files:
        sample_name = input_file.split('/')[-1].split('.')[0]
        df = pd.read_csv(input_file, sep=sep, skiprows=skiprows, nrows=nrows, index_col=index_col, header=header,names=col_names)
        
        if normalize != False:
            df['percent'] = df[normalize]/df[normalize].sum()*100
        if add_sample_name:
            df['sample_name'] = '_'.join(sample_name.split('_')[:-1])
            df['library_name'] = sample_name
        dfs.append(df)
    df = pd.concat(dfs, axis=axis)
    return df

In [ ]:
def annotate_df(df, labels=['sample_source','disease_stage','subject_id']):
    for label in labels:
        df[label] = [manifest.loc[sample_name, label] for sample_name in df.sample_name]
    return df
        

In [ ]:
def merge_fastq_data(df):
    dfs = []
    for g, group in df.groupby('sample_name'):
        group = pd.DataFrame(group.sum(numeric_only=True)).T
        for c in group:
            group[f'%{c}'] = group[c]/group['reads_before_filtering']*100
        group['sample_name'] = g
        dfs.append(group)
    df = pd.concat(dfs)
    return df

## I. fastp QC trimming results

In [ ]:
# fastp QC trimming results

fastq = glob_fastp_to_df(manifest, data_path=f'{batch_path}/fastp')
fastq = merge_fastq_data(fastq)
fastq = annotate_df(fastq)
fastq.sort_values(['sample_source','subject_id'], inplace=True)

fastq.to_excel(writer,sheet_name='fastq_filtering')
fig = px.bar(fastq, 
             x='subject_id', 
             y=['%passed_filter_reads','%low_quality_reads','%too_many_N_reads','%too_short_reads'],
             facet_col='sample_source',
             title='FASTQ QC summary'
            )

fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_xaxes(showline=True, linewidth=1, linecolor='grey',
                 showgrid=False,mirror=True,matches=None,
                 tickmode='linear',title='',tickangle=90,
                 tickfont=dict(size=10)
                )
fig.update_yaxes(showline=True, linewidth=0.8, linecolor='grey',
                 showgrid=True,mirror=True,gridcolor='lightgrey',gridwidth=0.1)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1],font=dict(size=10)))
fig = group_layout(fig,fastq, 'sample_source',jitter=False)

fig.show()

## II. GC distribution

In [ ]:
# Normalized GC coverage
%matplotlib inline

gc_data = glob_to_df(f'{batch_path}/picard_CollectMultipleMetrics/*.gc_bias.detail_metrics',
                     skiprows=6
                    )
gc_data = annotate_df(gc_data)


groups = gc_data.drop_duplicates('sample_source').shape[0]
fig, axs = plt.subplots(groups+2,1,figsize=(15,10),sharex=False,
                        gridspec_kw={"height_ratios":[1 for i in range(groups+1)]+[0.2]}
                       )

plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.01)
axs = axs.flatten()
axs[0] = sns.lineplot(x='GC', y='NORMALIZED_COVERAGE', hue='sample_source', data=gc_data[np.abs(gc_data['NORMALIZED_COVERAGE']-gc_data['NORMALIZED_COVERAGE'].mean()) <= (0.2*gc_data['NORMALIZED_COVERAGE'].std())].reset_index(), ci=None, ax=axs[0])
axs[0].hlines(1,0,100,color='black',linestyles='dashed')
axs[0].set_ylim(0,2)
axs[0].set_xlim(0,95)
axs[0].legend(loc = "upper right")
axs[0].set_title('Mean Normalized GC coverage',fontsize=16)

i = 1
dfs = []
for g, group in gc_data.groupby('sample_source'):
    y_values = []
    samples_names = []
    for s, sample in group.groupby('subject_id'):
        sample = sample[np.abs(sample['NORMALIZED_COVERAGE']-sample['NORMALIZED_COVERAGE'].mean()) <= (0.5*sample['NORMALIZED_COVERAGE'].std())]
        y = np.array(sample.NORMALIZED_COVERAGE.tolist())
        y_values.append(y)
        samples_names.append(s)
    matrix = np.matrix(y_values)
    matrix = pd.DataFrame(matrix)
    matrix['sample_name'] = samples_names
    matrix.set_index('sample_name', inplace=True)
    axs[i] = sns.heatmap(matrix,ax=axs[i],vmax=2, cbar=False)
    axs[i].set_yticklabels('')
    axs[i].set_ylabel(g)
    axs[i].set_xlim(0,95)
    axs[i].sharex(axs[0])
    i+=1
    dfs.append(matrix)

x = axs[-1].set_xlabel('% GC')
c = fig.colorbar(axs[-1].imshow(matrix,vmax=2), cax=axs[-1], orientation="horizontal",anchor=(0.9, 1.0))

matrix = pd.concat(dfs)
matrix = matrix.T
matrix['%GC'] = matrix.index.tolist()
matrix.to_excel(writer,sheet_name='GC distribution')

## III. AT and GC drop out

In [ ]:
# AT drop out
gc = glob_to_df(f'{batch_path}/picard_CollectMultipleMetrics/*gc_bias.summary_metrics',skiprows=6)
gc = annotate_df(gc)
gc.sort_values(['disease_stage','subject_id'], inplace=True)
gc.to_excel(writer,sheet_name='GC_bias')
fig = px.bar(gc, 
             x='subject_id', 
             y='AT_DROPOUT',
             color='sample_source',
             facet_col='sample_source',
             title='AT DROPOUT',
             height=350
            )

fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_xaxes(showline=True, linewidth=1, linecolor='grey',
                 showgrid=False,mirror=True,matches=None,
                 tickmode='linear',title='',tickangle=90,
                 tickfont=dict(size=10)
                )
fig.update_yaxes(showline=True, linewidth=0.8, linecolor='grey',
                 showgrid=True,mirror=True,gridcolor='lightgrey',gridwidth=0.1)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1],font=dict(size=10)))
fig = group_layout(fig,gc, 'sample_source',jitter=False)

fig.show()

fig = px.bar(gc, 
             x='subject_id', 
             y='GC_DROPOUT',
             color='sample_source',
             facet_col='sample_source',
             title='GC DROPOUT',
             height=350
            )

fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_xaxes(showline=True, linewidth=1, linecolor='grey',
                 showgrid=False,mirror=True,matches=None,
                 tickmode='linear',title='',tickangle=90,
                 tickfont=dict(size=10)
                )
fig.update_yaxes(showline=True, linewidth=0.8, linecolor='grey',
                 showgrid=True,mirror=True,gridcolor='lightgrey',gridwidth=0.1)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1],font=dict(size=10)))
fig = group_layout(fig,fastq, 'sample_source',jitter=False)

fig.show()


## IV. Insert size distributions

In [ ]:
# insert size distributions
insert_size = glob_to_df(f'{batch_path}/picard_CollectInsertSizeMetrics/*.insert_size_metrics.txt',
                     skiprows=11,normalize='All_Reads.fr_count'
                    )
insert_size = annotate_df(insert_size)

insert_size.sort_values(['disease_stage','subject_id'], inplace=True)

insert_size.to_excel(writer, sheet_name='fragment_lengths')


fig = px.line(insert_size, x='insert_size', y='percent', color='sample_source',hover_name='sample_source',
              line_group='subject_id',
              title='Mean fragment length distribution per sample type',
             )
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_xaxes(showline=True, linewidth=1, linecolor='grey',title='fragment length',
                 showgrid=False,mirror=True,matches=None)
fig.update_yaxes(showline=True, linewidth=0.8, linecolor='grey',title='percent',
                 showgrid=False,mirror=True,gridcolor='lightgrey',gridwidth=0.1)
fig.show()

In [ ]:
insert_size = glob_to_df(f'{batch_path}/picard_CollectInsertSizeMetrics/*.insert_size_metrics.txt',
                     skiprows=6,nrows=1)
insert_size = annotate_df(insert_size)


insert_size.sort_values(['disease_stage','subject_id'], inplace=True)

insert_size.to_excel(writer, sheet_name='fragment_length_stats')

fig = px.bar(insert_size, x='subject_id', y='MEDIAN_INSERT_SIZE',facet_col='sample_source',color='sample_source',
             title='Median fragment length per sample',height=350)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_xaxes(showline=True, linewidth=1, linecolor='grey',
                 showgrid=False,mirror=True,matches=None,
                 tickmode='linear',title='',tickangle=90,
                 tickfont=dict(size=10)
                )
fig.update_yaxes(showline=True, linewidth=0.8, linecolor='grey',range=(100,200),
                 showgrid=True,mirror=True,gridcolor='lightgrey',gridwidth=0.1)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1],font=dict(size=10)))
fig = group_layout(fig,insert_size, 'sample_source',jitter=False)
fig.show()


## V. Coverage

In [ ]:
depth = glob_to_df(f'{batch_path}/mosdepth/*.mosdepth.summary.txt')
depth = depth[depth.chrom=='total']
depth = annotate_df(depth)
depth.sort_values(['sample_source','subject_id'], inplace=True)
depth.to_excel(writer, sheet_name='depth_stats')
fig = px.bar(depth, x='subject_id', y='mean',facet_col='sample_source',
             color='sample_source',title='Mean coverage across the human genome',
             height=350
            )
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_xaxes(showline=True, linewidth=1, linecolor='grey',
                 showgrid=False,mirror=True,matches=None,
                 tickmode='linear',title='',tickangle=90,
                 tickfont=dict(size=10)
                )
fig.update_yaxes(showline=True, linewidth=0.8, linecolor='grey',
                 showgrid=True,mirror=True,gridcolor='lightgrey',gridwidth=0.1)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1],font=dict(size=10)))
fig = group_layout(fig,depth, 'sample_source',jitter=False)
fig.show()

In [ ]:
depth = glob_to_df(f'{batch_path}/mosdepth/*.mosdepth.global.dist.txt',header=None, col_names=['chrom','cov','percent'])
depth = depth[depth.chrom=='total']
depth = depth[depth.percent>0]
dfs = []
for g, group in depth.groupby('sample_name'):
    sample_source = manifest.loc[g,'sample_source']
    disease_stage = manifest.loc[g,'disease_stage']
    subject_id = manifest.loc[g,'subject_id']
    group['sample_source'] = sample_source
    group['disease_stage'] = disease_stage
    group['subject_id'] = subject_id
    dfs.append(group)
depth = pd.concat(dfs)
depth.sort_values(['disease_stage','subject_id'], inplace=True)



fig = px.line(depth, x='cov', y='percent', color='sample_source',hover_name='sample_source',
              line_group='subject_id',
              title='Whole genome coverage distribution',
             )
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_xaxes(showline=True, linewidth=1, linecolor='grey',title='fragment length',
                 showgrid=False,mirror=True,matches=None)
fig.update_yaxes(showline=True, linewidth=0.8, linecolor='grey',title='percent',
                 showgrid=False,mirror=True,gridcolor='lightgrey',gridwidth=0.1)
fig.show()


## VI. Mapping QC

In [ ]:
# picard metrics
picard = glob_to_df(f'{batch_path}/picard_CollectMultipleMetrics/*alignment_summary_metrics',skiprows=6,nrows=3)
picard = picard[picard.CATEGORY == 'PAIR']
picard = annotate_df(picard)
picard.sort_values(['sample_source','subject_id'], inplace=True)

picard.to_excel(writer, sheet_name='picard_metrics')

metrics = ['PCT_PF_READS_ALIGNED',
           'PCT_PF_READS_IMPROPER_PAIRS',
           'PCT_CHIMERAS',
           'PCT_SOFTCLIP',
           'PF_HQ_ERROR_RATE',
           'PF_INDEL_RATE',
           'PF_ALIGNED_BASES'
          ]
for metric in metrics:
        fig = px.bar(picard,x='subject_id',y=metric,
                     labels = dict(y='skjskdj'),
                     template='plotly',
                     height=350,color='sample_source',width=990,
                     facet_col='sample_source'
                    )



        fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',showlegend=False)
        fig.update_xaxes(showline=True, linewidth=1, linecolor='grey',
                         showgrid=False,mirror=True,title='',tickmode='linear',matches=None,
                        tickangle=90)
        fig.update_yaxes(showline=True, linewidth=0.8, linecolor='grey',title_font=dict(size=11),
                         showgrid=False,mirror=True,gridcolor='lightgrey',gridwidth=0.1)

        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1],font=dict(size=10)))
        fig = group_layout(fig,picard, 'sample_source')
        fig.show()

In [ ]:
writer.close()